In [3]:
import numpy as np
from sklearn.preprocessing import minmax_scale
from scipy.spatial.distance import cdist
from pyts.metrics import dtw

## Features

In [6]:
def x_value(line):
    line = line.split()
    return float(line[1])


def y_value(line):
    line = line.split()
    return float(line[2])


def pressure_value(line):
    line = line.split()
    return float(line[3])


def vx_value(prev_line, current_line):
    prev_line = prev_line.split()
    current_line = current_line.split()
    x1 = float(prev_line[1])
    x2 = float(current_line[1])
    t1 = float(prev_line[0])
    t2 = float(current_line[0])

    if t2-t1 == 0:
        return 0
    return (x2-x1)/(t2-t1)


def vy_value(prev_line, current_line):
    prev_line = prev_line.split()
    current_line = current_line.split()
    y1 = float(prev_line[2])
    y2 = float(current_line[2])
    t1 = float(prev_line[0])
    t2 = float(current_line[0])
    if t2-t1 == 0:
        return 0
    return (y2-y1)/(t2-t1)


# Compute the "time serie", the features for each line of the signature file
def compute_features_vector(signature, normalize=True):
    rep = []
    first = True
    for line in signature:
        if first:
            prev_line = line
            first = False
        feature_vector = [x_value(line), y_value(line), pressure_value(
            line), vx_value(prev_line, line), vy_value(prev_line, line)]
        rep.append(feature_vector)
        prev_line = line

    array = np.asarray(rep)

    if normalize:
        array = minmax_scale(array, feature_range=(0, 1), axis=0)

    return array

def compute_dtw(signature1, signature2, normalize=True):

    feature_vector_1 = compute_features_vector(signature1, normalize)
    feature_vector_2 = compute_features_vector(signature2, normalize)

    dist_matrix = cdist(feature_vector_1, feature_vector_2)

    dtw_cost = dtw(precomputed_cost=dist_matrix, dist="precomputed",
                   method="sakoechiba")

    return dtw_cost


In [7]:
f1 = open('data/enrollment/001-g-01.txt', 'r')
f2 = open('data/enrollment/001-g-02.txt', 'r')
print(compute_dtw(f1,f2))

18.37418347081205
